In [1]:
import os
import xarray as xr
import pandas as pd
import numpy as np

In [28]:
dir = "/home/rodr/buffalo/rodr/temp/"
files = os.listdir(dir)
ds = xr.open_dataset(dir + files[0])
ds

<xarray.Dataset> Size: 7TB
Dimensions:                   (phony_dim_0: 17520, phony_dim_1: 2018267)
Dimensions without coordinates: phony_dim_0, phony_dim_1
Data variables: (12/27)
    air_temperature           (phony_dim_0, phony_dim_1) float64 283GB ...
    alpha                     (phony_dim_0, phony_dim_1) float64 283GB ...
    aod                       (phony_dim_0, phony_dim_1) float64 283GB ...
    asymmetry                 (phony_dim_0, phony_dim_1) float64 283GB ...
    cld_opd_dcomp             (phony_dim_0, phony_dim_1) float64 283GB ...
    cld_press_acha            (phony_dim_0, phony_dim_1) float64 283GB ...
    ...                        ...
    surface_albedo            (phony_dim_0, phony_dim_1) float64 283GB ...
    surface_pressure          (phony_dim_0, phony_dim_1) float64 283GB ...
    time_index                (phony_dim_0) <U25 2MB ...
    total_precipitable_water  (phony_dim_0, phony_dim_1) float64 283GB ...
    wind_direction            (phony_dim_0, phony_dim_1) float64 283GB ...
    wind_speed                (phony_dim_0, phony_dim_1) float64 283GB ...
Attributes:
    full_version_record:  {"rex": "0.2.80", "pandas": "2.0.0", "numpy": "1.23...
    package:              rex
    version:              4.0.0

In [36]:
vars = ["ghi", "dni", "relative_humidity", "air_temperature", "wind_speed"]
ds = xr.open_dataset(dir + files[0])
#ds = ds.set_coords("time_index")
#ds["time_index"] = pd.to_datetime(ds["time_index"])
#ds = ds.swap_dims({"phony_dim_0": "time_index"})[vars]
ds["time_index"] = pd.to_datetime(ds["time_index"])
ds

MergeError: unable to determine if these variables should be coordinates or not in the merged result: {'time_index'}

In [22]:
ds

<xarray.Dataset> Size: 7TB
Dimensions:                   (phony_dim_0: 17520, phony_dim_1: 2018267,
                               time_index: 17520)
Coordinates:
  * time_index                (time_index) object 140kB 1640995200000000000 ....
Dimensions without coordinates: phony_dim_0, phony_dim_1
Data variables: (12/26)
    air_temperature           (phony_dim_0, phony_dim_1) float64 283GB ...
    alpha                     (phony_dim_0, phony_dim_1) float64 283GB ...
    aod                       (phony_dim_0, phony_dim_1) float64 283GB ...
    asymmetry                 (phony_dim_0, phony_dim_1) float64 283GB ...
    cld_opd_dcomp             (phony_dim_0, phony_dim_1) float64 283GB ...
    cld_press_acha            (phony_dim_0, phony_dim_1) float64 283GB ...
    ...                        ...
    ssa                       (phony_dim_0, phony_dim_1) float64 283GB ...
    surface_albedo            (phony_dim_0, phony_dim_1) float64 283GB ...
    surface_pressure          (phony_dim_0, phony_dim_1) float64 283GB ...
    total_precipitable_water  (phony_dim_0, phony_dim_1) float64 283GB ...
    wind_direction            (phony_dim_0, phony_dim_1) float64 283GB ...
    wind_speed                (phony_dim_0, phony_dim_1) float64 283GB ...
Attributes:
    full_version_record:  {"rex": "0.2.80", "pandas": "2.0.0", "numpy": "1.23...
    package:              rex
    version:              4.0.0

In [21]:
ds["time_index"]

<xarray.DataArray 'time_index' (time_index: 17520)> Size: 140kB
array([1640995200000000000, 1640997000000000000, 1640998800000000000, ...,
       1672525800000000000, 1672527600000000000, 1672529400000000000],
      dtype=object)
Coordinates:
  * time_index  (time_index) object 140kB 1640995200000000000 ... 16725294000...